In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

C:\Users\admin\Documents\ds4a


## Principal Libreries

In [2]:
import pandas as pd

In [3]:
from tools.generate_dic_data import apply_dic_data

In [4]:
from core.metadata.directory import PathDataRaw
from core.metadata.directory import PathOutput

In [5]:
from core.data_preparation.data_preparation import apply_data_preparation

## Read Data Dict

Check the function apply_dic_data in the folder core - data_preparation - data_preparation

In [6]:
df = apply_dic_data(PathDataRaw.data)

In [7]:
df['EARNINGS'] = df['EARNINGS'][['estate', 'county', 'year', 'total_med',
       'total_agri_fish_mine', 'agri_fish_hunt', 'construction',
       'manufacturing', 'wholesale_trade', 'retail_trade',
       'transport_warehouse_utilities', 'transport_warehouse',
       'fin_ins_realest', 'fin_ins', 'total_prof_sci_mgmt_admin',
       'prof_sci_tech', 'admin_sup', 'total_edu_health_social', 'edu_serv',
       'health_social', 'total_arts_ent_acc_food', 'acc_food_serv',
       'other_ser', 'pub_admin']]

## Cleanning df earnings

In [8]:
df['EARNINGS'].rename(columns={'estate':'state'}, inplace=True)

for col in ['state', 'county']:
    df['EARNINGS'][col] = df['EARNINGS'][col].str.upper()
    
df['EARNINGS']['county'] = df['EARNINGS'].county.str.split(',', expand=True)[0]
df['EARNINGS']['county'] = df['EARNINGS'].county.str.replace('COUNTY', '')
df['EARNINGS']['county'] = df['EARNINGS'].county.str.strip()
df['EARNINGS']['state'] = df['EARNINGS'].state.str.strip()

## Limit the years to analysis

In [9]:
cond_year_min = df['CHEMICALS'].year >= 2010
cond_year_max = df['CHEMICALS'].year <= 2016

df['CHEMICALS'] = df['CHEMICALS'][cond_year_min & cond_year_max]

df['CHEMICALS'] = df['CHEMICALS'].groupby(['state', 'county', 'chemical_species', 'contaminant_level',
                                   'unit_measurement', 'year'])[['value']].sum().reset_index()

## Standardize the series to works with the same units

In [10]:
for col in ['state', 'county']:
    df['CHEMICALS'][col] = df['CHEMICALS'][col].str.upper()
    
df['CHEMICALS'].value = df['CHEMICALS'].apply(lambda x: x.value/1000 if x.unit_measurement == 'micrograms/L' else x.value, axis=1)
df['CHEMICALS'].unit_measurement = 'mg/L'

df['CHEMICALS'] = df['CHEMICALS'].groupby(['state', 'county', 'chemical_species',
       'unit_measurement', 'year'])[['value']].sum().reset_index()

## Transform columns in rows and keep the series  'state', 'county', 'year'

In [11]:
df_modify = {}
lista_col = ['state', 'county', 'chemical_species', 'year', 'value']

for agente in df['CHEMICALS'].chemical_species.unique():
    df_modify[agente] = df['CHEMICALS'][
        df['CHEMICALS'].chemical_species == agente][lista_col]
    df_modify[agente].rename(columns={'value': f'{agente}_value'}, inplace=True)
    df_modify[agente].drop(columns=['chemical_species'], inplace=True)
    df_modify[agente].set_index(['state', 'county', 'year'], inplace=True)

## Join 'state', 'county', 'year' for every chemical agents

In [12]:
df['CHEMICALS'] = pd.concat([df_ for df_ in df_modify.values()], axis=1).reset_index()

In [13]:
df['CHEMICALS'].shape

(9877, 9)

## Cleanning earnings data to use later

In [14]:
df['EARNINGS'].drop(columns=['total_med','total_prof_sci_mgmt_admin',
       'total_edu_health_social', 'total_arts_ent_acc_food', 'total_agri_fish_mine',
       'fin_ins_realest', 'fin_ins', 'prof_sci_tech', 'admin_sup', 
                            ], inplace=True)

In [15]:
df['EARNINGS'] = df['EARNINGS'].groupby(['state', 'year'], as_index=False).sum()

In [16]:
df['CHEMICALS'] = df['CHEMICALS'].groupby(['state', 'year'], as_index=False).sum()

## Generate the principal dataframe, we called df_core

In [17]:
df_core = pd.merge(df['CHEMICALS'], df['EARNINGS'],
          how='inner', on=['state', 'year'])

## TEST FUNCTION

In [18]:
apply_data_preparation(saving=True, path_=PathOutput.output_data_core)

state  year  Arsenic_value  DEHP_value  Nitrates_value  \
0    CALIFORNIA  2010        4.99142     0.59460         4004.79   
1    CALIFORNIA  2011        5.31674     0.60256         3993.74   
2    CALIFORNIA  2012        5.21608     0.57244         4032.18   
3    CALIFORNIA  2013        4.37632     0.23550         3180.53   
4    CALIFORNIA  2014        4.50887     0.28650         3267.80   
..          ...   ...            ...         ...             ...   
176   WISCONSIN  2012        0.49201     0.04983         1039.38   
177   WISCONSIN  2013        0.11401     0.00553         1056.80   
178   WISCONSIN  2014        0.53515     0.05072          942.41   
179   WISCONSIN  2015        0.41888     0.02587         1086.56   
180   WISCONSIN  2016        0.08477     0.01371         1081.15   

     Uranium_value  Halo-Acetic Acid_value  Trihalomethane_value  \
0          6.19038                 0.00000               0.00000   
1          5.93616                 0.00000               0.00000   
2          6.21888                 0.00000               0.00000   
3          4.14461                 0.00000               0.00000   
4          4.78010                 0.00000               0.00000   
..             ...                     ...                   ...   
176        0.00010                 0.94536               2.29505   
177        0.00000                 1.69134               4.41584   
178        0.00300                 2.49718               6.03264   
179        0.00100                 2.36384               5.16290   
180        0.00256                 3.37457               6.91982   

     agri_fish_hunt  construction  manufacturing  wholesale_trade  \
0         1336430.0     2096147.0      2348584.0        2230525.0   
1         1310920.0     2020223.0      2358049.0        2066472.0   
2         1328323.0     1993915.0      2373978.0        2061156.0   
3         1352314.0     1999253.0      2413351.0        2063826.0   
4         1319145.0     1987001.0      2349487.0        2091815.0   
..              ...           ...            ...              ...   
176       1675651.0     2678880.0      2663736.0        2618021.0   
177       1734668.0     2694546.0      2707729.0        2678338.0   
178       1793580.0     2766838.0      2772115.0        2785115.0   
179       1827990.0     2840628.0      2786703.0        2800582.0   
180       1866595.0     2882477.0      2805394.0        2786698.0   

     retail_trade  transport_warehouse_utilities  transport_warehouse  \
0       1242316.0                      2544365.0            2254438.0   
1       1237424.0                      2530154.0            2179129.0   
2       1249100.0                      2551330.0            2183961.0   
3       1222170.0                      2592206.0            2213572.0   
4       1236386.0                      2580643.0            2172463.0   
..            ...                            ...                  ...   
176     1471711.0                      3026691.0            2817585.0   
177     1457819.0                      3035666.0            2826347.0   
178     1481636.0                      3095702.0            2847888.0   
179     1490326.0                      3111857.0            2864724.0   
180     1504640.0                      3126719.0            2872934.0   

      edu_serv  health_social  acc_food_serv  other_ser  pub_admin  
0    2095951.0      1870241.0       814223.0  1198683.0  3180744.0  
1    2046439.0      1841645.0       784664.0  1169546.0  3147641.0  
2    2036428.0      1840602.0       790054.0  1178427.0  3170028.0  
3    2065394.0      1880560.0       794986.0  1186634.0  3216174.0  
4    2067637.0      1901151.0       800591.0  1180954.0  3211526.0  
..         ...            ...            ...        ...        ...  
176  2566054.0      1997023.0       773233.0  1581447.0  3186080.0  
177  2557560.0      2013648.0       786173.0  1604017.0  3218413.0  
178  2643252.0      2077496.0       800896.0  160